<a href="https://colab.research.google.com/github/lblogan14/master_tensorflow_keras/blob/master/ch2_high_level_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

High-level libraries:
* TF Estimator - previously TF Learn
* TF Slim
* TFLearn
* PrettyTensor
* Sonnet

In [0]:
import tensorflow as tf
import numpy as np
import os

#TF Estimator - previously TF Learn
makes it simple to create and train models by
encapsulating the functionalities for training, evaluating, predicting and exporting.

TF Estimator is inspired from Scikit-Learn, providing four main functions on any kind of estimator:
* `estimator.fit()`
* `estimator.evaluate()`
* `estimator.predict()`
* `estimator.export()`

So far, TensorFlow provides the following pre-built estimators:
* `tf.contrib.learn.KMeansClustering`
* `tf.contrib.learn.DNNClassifier`
* `tf.contrib.learn.DNNRegressor`
* `tf.contrib.learn.DNNLinearCombinedRegressor`
* `tf.contrib.learn.DNNLinearCombinedClassifier`
* `tf.contrib.learn.LinearClassifier`
* `tf.contrib.learn.LinearRegressor`
* `tf.contrib.learn.LogisticRegressor`

##TF Estimator MNIST Example

In [21]:
from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph()

mnist = input_data.read_data_sets(os.path.join('.', 'mnist'), one_hot=False)

x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels

n_classes = 10
batch_size = 100
n_steps = 1000
learning_rate = 0.01

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [0]:
def model_fn(features, labels, mode):
    """ define the model function"""
    espec_op = tf.estimator.EstimatorSpec
    # features is a dict as per Estimator specifications
    x = features['images']
    # define the network
    layer_1 = tf.layers.dense(x, 32)
    layer_2 = tf.layers.dense(layer_1, 32)
    logits = tf.layers.dense(layer_2, n_classes)

    # define predicted classes
    predicted_classes = tf.argmax(logits, axis=1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        espec = espec_op(mode, predictions=predicted_classes)
    else:
        # define loss and optimizer
        entropy_op = tf.nn.sparse_softmax_cross_entropy_with_logits
        loss_op = tf.reduce_mean(entropy_op(logits=logits,
                                            labels=tf.cast(labels, dtype=tf.int32)
                                            )
                                 )
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())

        # define accuracy
        accuracy_op = tf.metrics.accuracy(labels=labels, predictions=predicted_classes)

        espec = espec_op(mode=mode,
                         predictions=predicted_classes,
                         loss=loss_op,
                         train_op=train_op,
                         eval_metric_ops={'accuracy': accuracy_op}
                         )

    return espec

In [23]:
# create estimator object
model = tf.estimator.Estimator(model_fn)

# train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_train},
    y=y_train,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True)
model.train(train_input_fn, steps=n_steps)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpcxg_xli1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fde4b3ea080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph wa

In [24]:
# evaluate model
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={'images':x_test},
                                                  y=y_test,
                                                  batch_size=batch_size,
                                                  shuffle=False)
model.evaluate(eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-06-04:46:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpcxg_xli1/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-06-04:46:10
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.8869, global_step = 1000, loss = 0.40368512
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmpcxg_xli1/model.ckpt-1000


{'accuracy': 0.8869, 'global_step': 1000, 'loss': 0.40368512}

It is totally fine if you cannot fully understand some of the codes. They will be much more understandable in the next few chapters.

#TF Slim
a lightweight library built on top of TensorFlow core for defining and training mdoels.

`tf.contrib.slim`

check TF Slim installation:

    python3 -c 'import tensorflow.contrib.slim as slim; eval =
    slim.evaluation.evaluate_once'

TF Slim provides several modules that can be picked and applied independently and mixed
with other TensorFlow.

TF Slim module | Module description
--- | ---
arg_scope | Provides a mechanism to apply elements to all graph nodes defined under a scope.
layers | Provides several different kinds of layers such as `fully_connected`, `conv2d`, and many more.
losses | Provides loss functions for training the optimizer
learning | Provides functions for training the models
evaluation | Provides evaluation functions
metrics | Provides metrics functions to be used for evaluating the models
regularizers | Provides functions for creating regularization methods
variables | Provides functions for variable creation
nets | Provides various pre-built and pre-trained models such as VGG16, InceptionV3, ResNet

##TF Slim MNIST Example,

In [25]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import slim

tf.reset_default_graph()

n_classes = 10
n_steps = 1000

# get the data
mnist = input_data.read_data_sets(os.path.join('.', 'mnist'), one_hot=True)

X_train = mnist.train.images
X_train = tf.convert_to_tensor(X_train)
Y_train = mnist.train.labels
Y_train = tf.convert_to_tensor(Y_train)

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [0]:
def mlp(x):
  net = slim.fully_connected(x, 32, scope='fc1')
  net = slim.dropout(net, 0.5, scope='dropout1')
  net = slim.fully_connected(net, 32, scope='fc2')
  net = slim.dropout(net, 0.5, scope='dropout2')
  net = slim.fully_connected(net, n_classes, activation_fn=None, scope='fc3')
  return net

In [0]:
# define model
logits = mlp(X_train)

# define the loss functions and get the total loss
loss = tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=Y_train)
total_loss = tf.losses.get_total_loss()

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

train_op = slim.learning.create_train_op(total_loss, optimizer)

In [28]:
# train model
final_loss = slim.learning.train(train_op,
                                logdir='./slim_logs',
                                number_of_steps=n_steps,
                                log_every_n_steps=100)
print('final loss={}'.format(final_loss))

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Saving checkpoint to path ./slim_logs/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global step 100: loss = 2.2426 (0.285 sec/step)
INFO:tensorflow:global step 200: loss = 2.1616 (0.281 sec/step)
INFO:tensorflow:global step 300: loss = 2.0764 (0.281 sec/step)
INFO:tensorflow:global step 400: loss = 1.9907 (0.285 sec/step)
INFO:tensorflow:global step 500: loss = 1.9062 (0.284 sec/step)
INFO:tensorflow:global step 600: loss = 1.8290 (0.284 sec/step)
INFO:tensorflow:global step 700: loss = 1.7650 (0.287 sec/step)
INFO:tensorflow:global step 800: loss = 1.6980 (0.286 sec/step)
INFO:tensorflow:global step 900: loss = 1.6325 (0.284 sec/step)
INFO:tensorflow:global step 1000: loss = 1.5830 (0.282 sec/step)
INFO:tensorflow:Stopping Traini

#TFLearn
`pip3 install tflearn`

##Creating the TFlearn Layers
1. Input layer

    `input_layer = tflearn.input_data(shape=[None, num_inputs])`
    
2. Further layers
    
    `layer1 = tflearn.fully_connected(input_layer, 10, activation='relu')`
    
    `layer2 = tflearn.fully_connected(layer1, 10, activation='relu')`
    
3. Output layer

    `output = tflearn.fully_connected(layer2, n_classes, activation='softmax')`
    
4. Create the final network from the estimator layer such as `regression`:

    `net = tflearn.regression(output,optimizer='adam', metric=tflearn.metrics.Accuracy(), loss='categorical_crossentropy')`

##TFLearn core layers
In the `tflearn.layers.core` module,

Layer class | Description
--- | ---
`input_data` | This layer is used to specify the input layer for the neural network.
`fully_connected` | This layer is used to specify a layer where all the neurons are connected to all the neurons in the previous layer.
`dropout` | This layer is used to specify the dropout regularization. The input elements are scaled by `1/keep_prob` while keeping the expected sum unchanged.
`custom_layer` | This layer is used to specify a custom function to be applied to the input. This class wraps our custom function and presents the function as a layer.
`reshape` | This layer reshapes the input into the output of specified shape.
`flatten` | This layer converts the input tensor to a 2D tensor.
`activation` | This layer applies the specified activation function to the input tensor.
`single_unit` | This layer applies the linear function to the inputs.
`highway` | This layer implements the fully connected highway function.
`one_hot_encoding` | This layer converts the numeric labels to their binary vector one-hot encoded representations.
`time_distributed` | This layer applies the specified function to each time step of the input tensor.
`multi_target_data` | This layer creates and concatenates multiple placeholders, specifically used when the layers use targets from multiple sources

##TFLearn convolutional layers
In the `tflearn.layers.conv` module,

Layer class | Description
--- | ---
`conv_1d` | This layer applies 1D convolutions to the input data
`conv_2d` | This layer applies 2D convolutions to the input data
`conv_3d` | This layer applies 3D convolutions to the input data
`conv_2d_transpose` | This layer applies transpose of conv2_d to the input data
`conv_3d_transpose` | This layer applies transpose of conv3_d to the input data
`atrous_conv_2d` | This layer computes a 2-D atrous convolution
`grouped_conv_2d` | This layer computes a depth-wise 2-D convolution
`max_pool_1d` | This layer computes 1-D max pooling
`max_pool_2d` | This layer computes 2D max pooling
`avg_pool_1d` | This layer computes 1D average pooling
`avg_pool_2d` | This layer computes 2D average pooling
`upsample_2d` | This layer applies the row and column wise 2-D repeat operation upscore_layer This layer implements the upscore as specified in http://arxiv.org/abs/1411.4038
`global_max_pool` | This layer implements the global max pooling operation
`global_avg_pool` | This layer implements the global average pooling operation
`residual_block` | This layer implements the residual block to create deep residual networks
`residual_bottleneck` | This layer implements the residual bottleneck block for deep residual networks
`resnext_block` | This layer implements the ResNeXt block

##TFLearn recurrent layers
In the `tflearn.layers.recurrent` module,

Layer class | Description
--- | ---
`simple_rnn` | This layer implements the simple recurrent neural network model
`bidirectional_rnn` | This layer implements the bi-directional RNN model
`lstm` | This layer implements the LSTM model
`gru` | This layer implements the GRU model

##TFLearn normalization layers
In the `tflearn.layers.normalization` module,

Layer class | Description
--- | ---
`batch_normalization` | This layer normalizes the output of activations of previous layers for each batch
`local_response_normalization` | This layer implements the LR normalization
`l2_normalization` | This layer applies the L2 normalization to the input tensors

##TFLearn embedding layers
In the `tflearn.layers.embedding_ops` module,

Layer class | Description
--- | ---
`embedding` | This layer implements the embedding function for a sequence of integer IDs or floats

##TFLearn merge layers
In the `tflearn.layers.merge_ops` module,

Layer class | Description
--- | ---
`merge_outputs` | This layer merges the list of tensors into a single tensor, generally used to merge the output tensors of the same shape
`merge` | This layer merges the list of tensors into a single tensor; you can specify the axis along which the merge needs to be done

##TFLearn estimator layers
In the `tflearn.layers.estimator` module,

Layer class | Description
--- | ---
`regression` | This layer implements the linear or logistic regression

For regression, you can also specify the optimizer and the loss and metric functions.

For optimizer functions in the `tflearn.optimizers` module,
* `SGD`
* `RMSprop`
* `Adam`
* `Momentum`
* `AdaGrad`
* `Ftrl`
* `AdaDelta`
* `ProximalAdaGrad`
* `Nesterov`

or you can create custom optimizer using the base class `tflearn.optimizers.Optimizer`

For metric functions in the `tflearnl.metrics` module,
* `Accuracy` or `accuracy_op`
* `Top_k` or `top_k_op`
* `R2` or `r2_op`
* `WeightedR2` or `weighted_r2_op`
* `binary_accuracy_op`

Or custom metrics, `tflearn.metrics.Metric` base class

For loss functions in the `tflearn.objectives` module,
* `softymax_categorical_crossentropy`
* `categorical_crossentropy`
* `binary_crossentropy`
* `weighted_crossentropy`
* `mean_square`
* `hinge_loss`
* `roc_auc_score`
* `weak_cross_entropy_2d`

For activation functions in the `tflearn.activations` module,
* `linear`
* `tanh`
* `sigmoid`
* `softmax`
* `softplus`
* `softsign`
* `relu`
* `relu6`
* `leaky_relu`
* `prelu`
* `elu`
* `crelu`
* `selu`

##Creating the TFLearn Model
`model = tflearn.DNN(net)`
##Types of TFLearn Models
* `DNN` model, multilayer perceptron from the network that you have created from the layers
* `SequenceGenerator` model, allows you to create a deep neural network that can generate sequences

##Training the TFLearn Model
`model.fit(X_train, Y_train, n_epoch=n_epochs, batch_size=batch_size, show_metric=True, run_id='dense_model')`
##Using the TFLearn Model
`score = model.evaluate(X_test, Y_test)`

##TFLearn MNIST Example,

In [3]:
!pip3 install tflearn

    100% |████████████████████████████████| 102kB 3.2MB/s 
  Running setup.py bdist_wheel for tflearn ... - \ done
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [4]:
import tflearn
import tflearn.datasets.mnist as mnist
batch_size = 100
n_classes = 10
n_epochs = 10

X_train, Y_train, X_test, Y_test = mnist.load_data(data_dir=os.path.join('.', 'mnist'), one_hot=True)

Succesfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/train-images-idx3-ubyte.gz
Succesfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/train-labels-idx1-ubyte.gz
Succesfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Succesfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [5]:
# Build deep neural network
input_layer = tflearn.input_data(shape=[None, 784])
layer1 = tflearn.fully_connected(input_layer, 10, activation='relu')
layer2 = tflearn.fully_connected(layer1, 10, activation='relu')
output = tflearn.fully_connected(layer2, n_classes, activation='softmax')

net = tflearn.regression(output, 
                         optimizer='adam', 
                         metric=tflearn.metrics.Accuracy(),
                         loss='categorical_crossentropy'
                         )
model = tflearn.DNN(net)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
# Train DNN
model.fit(X_train,
          Y_train,
          n_epoch=n_epochs,
          batch_size=batch_size,
          show_metric=True,
          run_id='dense_model')

Training Step: 5499  | total loss: 0.37611 | time: 5.368s
| Adam | epoch: 010 | loss: 0.37611 - acc: 0.9101 -- iter: 54900/55000
Training Step: 5500  | total loss: 0.35741 | time: 5.380s
| Adam | epoch: 010 | loss: 0.35741 - acc: 0.9151 -- iter: 55000/55000
--


In [7]:
# Evaluate the DNN
score = model.evaluate(X_test, Y_test)
print('Test accuracy:', score[0])

Test accuracy: 0.9201


#PrettyTensor
provides a thin wrapper on top of TensorFlow.

The objects provided by PrettyTensor support a chainable syntax to define neural networks.

PrettyTensor offers a very lightweight and extensible interface in the form of a method
named `apply()`. Any additional function can be chained to PrettyTensor objects using the
`.apply(function, arguments)` method. PrettyTensor will call the `function` and
supply the current tensor as the first argument to the `function`.

##MNIST example,

In [8]:
!pip3 install prettytensor

    100% |████████████████████████████████| 276kB 5.3MB/s 


In [0]:
tf.reset_default_graph()

In [23]:
from __future__ import division
import prettytensor as pt
from prettytensor.tutorial import data_utils

data_utils.WORK_DIRECTORY = os.path.join('.', 'mnist')

# get the data
X_train, Y_train = data_utils.mnist(training=True)
X_test, Y_test = data_utils.mnist(training=False)

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [0]:
# define hyperparameters

batch_size = 100     # number of samples that would be used to learn the parameters in a batch
n_classes = 10       # number of outputs, i.e. digits 0 to 9
n_epochs = 10        # number of ietrations for learning the parameters
n_batches = int(X_train.shape[0] / batch_size)
n_samples_in_train_batch = 60000 // batch_size
n_samples_in_test_batch = 10000 // batch_size

In [0]:
# define inputs and outputs

X = tf.placeholder(tf.float32, [batch_size, 28, 28, 1])
Y = tf.placeholder(tf.float32, [batch_size, 10])


In [0]:
# define the model

pretty_X = pt.wrap(X)

model = (pretty_X.flatten().
         fully_connected(10).
         softmax_classifier(n_classes, labels=Y))


In [0]:
# define evaluator (metrics), optimizer and training functions

evaluator = model.softmax.evaluate_classifier(Y)
optimizer = tf.train.GradientDescentOptimizer(0.1)
trainer = pt.apply_optimizer(optimizer, losses=[model.loss])

runner = pt.train.Runner()

In [0]:
with tf.Session() as tfs:
    for epoch in range(0, n_epochs):
        # shuffle the training data
        X_train, Y_train = data_utils.permute_data((X_train, Y_train))

        runner.train_model(
            trainer,
            model.loss,
            n_samples_in_train_batch,
            feed_vars=(X, Y),
            feed_data=pt.train.feed_numpy(batch_size, X_train, Y_train),
            print_every=600
        )

        score = runner.evaluate_model(
            evaluator,
            n_samples_in_test_batch,
            feed_vars=(X, Y),
            feed_data=pt.train.feed_numpy(batch_size, X_test, Y_test)
        )

        print('Accuracy after {} epochs {} \n'.
              format(epoch + 1, score[0]))

#Sonnet
`pip3 install dm-sonnet`

The modules are defined as sub-classes of the abstract class `sonnet.AbstractModule`.

Modules | Description
--- | ---
Basic modules | `AddBias, BatchApply, BatchFlatten, BatchReshape, FlattenTrailingDimensions, Linear, MergeDims, SelectInput, SliceByDim, TileByDim,` and `TrainableVariable`
Recurrent modules | `DeepRNN, ModelRNN, VanillaRNN, BatchNormLSTM, GRU,` and `LSTM`
Recurrent + ConvNet modules | `Conv1DLSTM` and `Conv2DLSTM`
ConvNet modules | `Conv1D, Conv2D, Conv3D, Conv1DTranspose, Conv2DTranspose, Conv3DTranspose, DepthWiseConv2D, InPlaneConv2D,` and `SeparableConv2D`
ResidualNets | `Residual, ResidualCore,` and `SkipConnectionCore`
Others | `BatchNorm, LayerNorm, clip_gradient,` and `scale_gradient`

##Sonnet MNIST Example,

In [0]:
tf.reset_default_graph()

In [29]:
!pip3 install dm-sonnet

    100% |████████████████████████████████| 634kB 6.9MB/s 


In [33]:
!pip3 install tensorflow-probability-gpu

In [0]:
import sonnet as snt

from tensorflow.examples.tutorials.mnist import input_data

tf.logging.set_verbosity(tf.logging.INFO)

In [0]:
class MNIST(snt.AbstractModule):

    def __init__(self, mnist_part, batch_size, name='MNIST'):

        super(MNIST, self).__init__(name=name)

        self._X = tf.constant(mnist_part.images, dtype=tf.float32)
        self._Y = tf.constant(mnist_part.labels, dtype=tf.float32)
        self._batch_size = batch_size
        self._M = mnist_part.num_examples

    def _build(self):
        idx = tf.random_uniform([self._batch_size], 0, self._M, tf.int64)
        X = tf.gather(self._X, idx)
        Y = tf.gather(self._Y, idx)
        return X, Y


In [0]:
class MLP(snt.AbstractModule):
    def __init__(self, output_sizes, name='mlp'):
        super(MLP, self).__init__(name=name)

        self._layers = []

        for output_size in output_sizes:
            self._layers.append(snt.Linear(output_size=output_size))

    def _build(self, X):

        # add the input layer
        model = tf.sigmoid(self._layers[0](X))

        # add hidden layers
        for i in range(1, len(self._layers) - 1):
            model = tf.sigmoid(self._layers[i](model))

        # add output layer
        model = tf.nn.softmax(self._layers[len(self._layers) - 1](model))

        return model


In [0]:
batch_size = 100
n_classes = 10
n_epochs = 10

mnist = input_data.read_data_sets(os.path.join('.', 'mnist'),
                                  one_hot=True
                                  )
train = MNIST(mnist.train, batch_size=batch_size)
test = MNIST(mnist.test, batch_size=batch_size)

X_train, Y_train = train()
X_test, Y_test = test()

model = MLP([20, n_classes])

Y_train_hat = model(X_train)
Y_test_hat = model(X_test)


In [0]:
def loss(Y_hat, Y):
    return -tf.reduce_sum(Y * tf.log(Y_hat))


L_train = loss(Y_train_hat, Y_train)
L_test = loss(Y_test_hat, Y_test)

optimizer = tf.train.GradientDescentOptimizer(
    learning_rate=0.01).minimize(L_train)

In [0]:
with tf.Session() as tfs:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        loss_val, _ = tfs.run((L_train, optimizer))
        print('Epoch : {} Training Loss : {}'.format(epoch, loss_val))

    loss_val = tfs.run(L_test)
    print('Test loss : {}'.format(loss_val))

#Summary
High-Level Library | Documentation Link | Source Code Link | pip3 install package
--- | --- | --- | ---
TF Estimator | https://www.tensorflow.org/get_started/estimator | https://github.com/tensorflow/tensorflow/tree/master/tensorflow/python/estimator | preinstalled with TensorFlow
TF Slim | https://github.com/tensorflow/tensorflow/tree/r1.4/tensorflow/contrib/slim | https://github.com/tensorflow/tensorflow/tree/r1.4/tensorflow/contrib/slim | preinstalled with TensorFlow
TFLearn | http://tflearn.org/ | https://github.com/tflearn/tflearn | tflearn
PrettyTensor | https://github.com/google/prettytensor/tree/master/docs | https://github.com/google/prettytensor | prettytensor
Sonnet | https://deepmind.github.io/sonnet/ | https://github.com/deepmind/sonnet | dm-sonnet